In [1]:
import os
import ast
from datetime import date

from netCDF4 import Dataset
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import contextily as ctx
from pyproj import Transformer
import plotly.express as px

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, Range1d, PanTool, WheelZoomTool, BoxSelectTool, ResetTool, HoverTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis256

import utils

In [2]:
list(range(0, 24, 6))

[0, 6, 12, 18]

In [ ]:
lakes_gdf = utils.get_hrrrx_lake_output(date(2020, 5, 20), date(2020, 6, 20), cycle_hours=list(range(0, 24, 6)), pred_hours=[0])

./data/hrrrX/sfc/20200601/hrrrX.t18z.wrfsfcf00.grib2: 1KB [00:00, 14.32KB/s]
./data/hrrrX/sfc/20200602/hrrrX.t00z.wrfsfcf00.grib2: 140226KB [00:06, 20320.55KB/s]                            
./data/hrrrX/sfc/20200602/hrrrX.t06z.wrfsfcf00.grib2: 126349KB [00:06, 19336.42KB/s]                            
./data/hrrrX/sfc/20200602/hrrrX.t12z.wrfsfcf00.grib2: 133734KB [00:06, 21462.44KB/s]                            
./data/hrrrX/sfc/20200602/hrrrX.t18z.wrfsfcf00.grib2: 140940KB [00:07, 18900.07KB/s]                            
./data/hrrrX/sfc/20200603/hrrrX.t00z.wrfsfcf00.grib2: 138269KB [00:09, 14081.80KB/s]                            
./data/hrrrX/sfc/20200603/hrrrX.t06z.wrfsfcf00.grib2: 126166KB [00:06, 18257.19KB/s]                            
./data/hrrrX/sfc/20200603/hrrrX.t12z.wrfsfcf00.grib2: 135241KB [00:09, 13557.33KB/s]                            
./data/hrrrX/sfc/20200603/hrrrX.t18z.wrfsfcf00.grib2: 141129KB [00:07, 19498.73KB/s]                            
./data/hrrrX/sfc/20

In [ ]:
# using manually-created bounding box for seneca lake
coords = [
    (-76.998076, 42.877521), 
    (-76.860185, 42.881791), 
    (-76.823285, 42.375953), 
    (-76.968944, 42.367344)
]

seneca_lake_bounds = gpd.GeoSeries([Polygon(coords)], crs='epsg:4326')
seneca_lake_bounds = seneca_lake_bounds.to_crs(epsg=3857).unary_union

In [ ]:
lakes_gdf = lakes_gdf.assign(**{ 'seneca_lake': lakes_gdf.within(seneca_lake_bounds) })
seneca_lake_points = lakes_gdf[lakes_gdf['seneca_lake']]
seneca_lake_points.head()

In [ ]:
fig = px.line(
    seneca_lake_points,
    x='fcst_datetime',
    y='water_temp',
    color='idx'
)
fig.update_layout(
    title="HRRRX/CLM 0hr-Preds for Seneca Lake from 20200620-20200520 Fcst Cycles",
    xaxis_title="Forcast Cycle",
    yaxis_title="Water Temp (C)",
    xaxis = {
        'dtick': 3600000.0
    }
)
fig.show()